In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy
df = pd.read_csv("candidates_choice_vows.csv",index_col=0)
df.head()

,Horodateur,Prénom,First_Choice,Reason1,Second_Choice,Reason2,Third_Choice,Teammates,Github
0,02/01/2023 14:53:21,0,3,4,2,0,5,1,0
1,03/01/2023 09:57:23,1,3,4,4,4,1,NaN,0
2,03/01/2023 11:11:46,2,2,2,1,2,5,NaN,0
3,03/01/2023 11:53:12,3,3,4,4,2,5,NaN,3
4,04/01/2023 11:24:51,4,3,4,5,3,4,NaN,1


In [22]:
df

,Horodateur,Prénom,First_Choice,Reason1,Second_Choice,Reason2,Third_Choice,Teammates,Github
0,02/01/2023 14:53:21,0,3,4,2,0,5,1,0
1,03/01/2023 09:57:23,1,3,4,4,4,1,NaN,0
2,03/01/2023 11:11:46,2,2,2,1,2,5,NaN,0
3,03/01/2023 11:53:12,3,3,4,4,2,5,NaN,3
4,04/01/2023 11:24:51,4,3,4,5,3,4,NaN,1
5,04/01/2023 13:56:21,5,2,3,4,2,5,NaN,1
6,04/01/2023 15:38:00,6,3,4,5,4,4,4 and 9,0
7,05/01/2023 01:10:24,7,4,2,1,2,2,NaN,1
8,05/01/2023 09:39:14,8,1,3,5,2,3,NaN,3
9,06/01/2023 09:01:35,9,4,5,3,5,5,NaN,2


In [2]:
candidate_vows ={ candidate :  [df.loc[candidate,'First_Choice'],df.loc[candidate,'Second_Choice'],df.loc[candidate,'Third_Choice']]   for candidate in df.index}

In [3]:
projet_vows = {i : list(np.arange(0,10)) for i in range(1,6)}

[np.random.shuffle(v) for k,v in projet_vows.items()]
projet_vows

{1: [3, 8, 5, 7, 9, 2, 6, 0, 4, 1],
 2: [0, 6, 2, 1, 3, 7, 8, 4, 5, 9],
 3: [3, 2, 0, 6, 5, 8, 7, 1, 9, 4],
 4: [6, 4, 2, 0, 1, 5, 8, 3, 7, 9],
 5: [8, 1, 9, 5, 6, 2, 0, 7, 3, 4]}

In [4]:
candidate_vows 

{0: [3, 2, 5],
 1: [3, 4, 1],
 2: [2, 1, 5],
 3: [3, 4, 5],
 4: [3, 5, 4],
 5: [2, 4, 5],
 6: [3, 5, 4],
 7: [4, 1, 2],
 8: [1, 5, 3],
 9: [4, 3, 5]}

In [55]:

[value for value in candidate_vows[1] if value in [3,5,4]]

candidate_vows[1].index(3)

0

In [62]:
projet_vows[5].index(5)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [44]:
list(projet_vows.keys())

[1, 2, 3, 4, 5]

In [30]:
np.random.sample(np.arange(0,10))  

array([], shape=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), dtype=float64)

In [5]:
def gale_shapley(candidate_vows:dict,projects_vows:dict)->dict:
    """
    A simple implementation of the gale-Shapley Algorithm
    """
    waiting_list = []
    matchings= {}
    projects_available = {candidate:projects_vows.keys() for candidate in candidate_vows.keys()}
    while len(waiting_list)<len(candidate_vows.keys()) :
        for candidate in list(candidate_vows.keys()):
            if candidate not in waiting_list:
                #filtering only available projects among the candidates'vows and select the first one
                available_vows =[proj for proj in candidate_vows[candidate] if proj in projects_available[candidate]]
                
                if len(available_vows)>0:
                    best_choice= available_vows[0]
                    #Recording the position of the best available vows among the candidates'vows
                #as well as the position of the proposed candidate among the best project's vows
                    matchings[(candidate, best_choice)]= (candidate_vows[candidate].index(best_choice),
                    projects_vows[best_choice].index(candidate))
        # if a project have more than one proposals 
        # it will choose the best option
        overlays = Counter([key[1] for key in matchings.keys()])
        for proj in overlays.keys():
            if overlays[proj]>1:
                # pairs to drop from proposals
                pairs_to_drop = sorted({pair: matchings[pair] for pair in matchings.keys() 
                if proj in pair}.items(), 
                   key=lambda x: x[1][1]
                  )[1:]
            # if man was rejected by woman
            # there is no pint for him to make proposal 
            # second time to the same woman
                for p_to_drop in pairs_to_drop:
                    del matchings[p_to_drop[0]]
                    _proj = copy(list(projects_available[p_to_drop[0][0]]))
                    _proj.remove(p_to_drop[0][1])
                    projects_available[p_to_drop[0][0]] = _proj
        # candidate who successfully created pairs must be added to the waiting list 
        waiting_list = [candidate[0] for candidate in matchings.keys()]   
        print("waiting_list=",waiting_list)
    return(matchings)
                

In [6]:
gale_shapley({0:[1,2,0],1:[2,0,1],2:[1,0,2]},{1:[2,1,0],0:[1,2,0],2:[2,0,1]})

waiting_list= [2]
waiting_list= [2, 0, 1]


{(2, 1): (0, 0), (0, 2): (1, 1), (1, 0): (1, 0)}